In [53]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import numpy as np
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [54]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            

# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s


import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [55]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10853 sentence pairs
Counting words...
Counted words:
fra 4489
eng 2925
['vous etes maigrichon .', 'you re skinny .']


In [56]:
pairs[419]

['nous sommes seules .', 'we re alone .']

### The above is the data preparation stage(for single batch). No code should be changed. 

## 1. First test out with no batch size

### 1.1 Encoder with no batch

In [57]:
#Here for the constant definition
MAX_SENTENCE_LENGTH = 10
hidden_size = 256
SOS_token = 0
EOS_token = 1
max_length = 10


In [58]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1,1,-1) # B x 1 x H
        #print('The embeddded size is {}'.format(embedded.size()))
        #When feeded in batch sizes, the size will be B x 1 x H, one token at a time
        output = embedded##Need changes if doing the batch size
        output, hidden = self.gru(output, hidden)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1,1,self.hidden_size, device = device)

In [59]:
#For 1 batch size, prepare the test data
source_list = [pairs[i][0] for i in range(10)]
output_list = [pairs[i][1] for i in range(10)]

#Here they are strings:
input_tensor = source_list[0]
output_tensor = output_list[0]

#Transfer to tensor
input_tensor = tensorFromSentence(input_lang, input_tensor)
output_tensor = tensorFromSentence(output_lang, output_tensor)


In [60]:
encoder = EncoderRNN(input_lang.n_words, hidden_size)
if device == torch.device('cuda'):
    encoder.cuda()
else:
    encoder.cpu()

In [61]:
input_length = input_tensor.size(0)
output_length = output_tensor.size(0)

encoder_outputs = torch.zeros(MAX_SENTENCE_LENGTH, 
                              encoder.hidden_size, 
                              device = device)#10x256
encoder_hidden = encoder.initHidden()#1x1x256

#Encode the sentence one index at a time
for ei in range(input_length):
    encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
    encoder_outputs[ei] = encoder_output[0,0]

### 1.2 Decoder with Attention
The decoder has already been batchified. Therefore 
if feeding 1 sentence at a time make sure unsqueeze the batch dimension

In [62]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers = 1, dropout_p = 0.1):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size#vocab size of the output lang
        self.n_layers = n_layers
        self.dropout_p = dropout_p

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.dropout = nn.Dropout(dropout_p)
        self.attn1 = nn.Linear(hidden_size, hidden_size)
        self.attn2 = nn.Linear(hidden_size, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))
        self.gru = nn.GRU(hidden_size*2, hidden_size, n_layers, dropout = dropout_p)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, word_input, last_hidden, encoder_outputs):
        batch_size = encoder_outputs.size(0)
        word_embedded = self.embedding(word_input).view(1,1,-1)
        word_embedded = self.dropout(word_embedded)

        #last hidden = B x 1 x 256
        #encoder outputs = B x 10 x 256

        #Scoring function
        last_hidden_repeated = last_hidden.repeat(1,encoder_outputs.size(1),1)
        attn_weights = F.tanh(torch.add(self.attn1(last_hidden_repeated),self.attn2(encoder_outputs)))
        v = self.v.unsqueeze(0).repeat(batch_size,1).unsqueeze(1)
        beta = v.bmm(attn_weights.transpose(1,2)) #batch x 1 x len
        alpha = F.softmax(beta.squeeze(1)).unsqueeze(1)#batch x 1 x len
        #End of scoring function
        
        context = alpha.bmm(encoder_outputs)#1 x 1 x 10 1 x 10 x 256 -> 1x 1 x 256
        rnn_input = torch.cat((word_embedded, context),2)#batch x 1 x 2hidden
        output, hidden = self.gru(rnn_input, last_hidden)#Hidden layer: layer x Bathc x Hidd

        output = output.squeeze(0)
        output = F.log_softmax(self.out(output))

        return output, hidden, alpha

# For single interations only, encoer_outputs already unsqueezed 

In [65]:
encoder_outputs.size()
# encoder_outputs = encoder_outputs.unsqueeze(0)

torch.Size([1, 10, 256])

In [66]:
attn_decoder = AttnDecoderRNN(hidden_size, output_lang.n_words)
if device == torch.device('cuda'):
    attn_decoder.cuda()
else:
    attn_decoder.cpu()

In [67]:
decoder_input = torch.tensor([[SOS_token]], device = device)
decoder_hidden = encoder_hidden#1x1x256

In [68]:
for di in range(output_length):
    decoder_output, decoder_hidden, decoder_attntion = attn_decoder(decoder_input, 
                                                     decoder_hidden, encoder_outputs)
    decoder_input = output_tensor[di]

/home/yz4499/miniconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/yz4499/miniconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


## Encoder outputs remained unchanged
## End of the decoder and encoder session
code shall not be changed. 
two sentence has already been encoded and now we
aim to generate best translation

## During the train, will be the inference state
two methods: greedy and beam search

# First greedy search: training and evaluations

In [77]:
#Greedy search
#DOnt execute these
#Training process:
teacher_forcing_ratio = 0.6
use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

if use_teacher_forcing:
    # Teacher forcing: Feed the target as the next input
    for di in range(output_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        loss += criterion(decoder_output, target_tensor[di])
        decoder_input = target_tensor[di]  # Teacher forcing

else:
    # Without teacher forcing: use its own predictions as the next input
    for di in range(output_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        #topv is the score, topi is the index of the highest score
        topv, topi = decoder_output.topk(1)

        decoder_input = topi.squeeze().detach()  # detach from history as input

        loss += criterion(decoder_output, target_tensor[di])
        if decoder_input.item() == EOS_token:
            break

NameError: name 'decoder' is not defined

In [71]:
#initialize the decoder_hidden here
decoder_input = torch.tensor([[SOS_token]], device = device)
decoder_hidden = encoder_hidden#1x1x256

In [72]:
#Evaluations:
decoded_words = []
decoder_attentions = torch.zeros(max_length, max_length)

for di in range(max_length):
    decoder_output, decoder_hidden, decoder_attention = attn_decoder(decoder_input, decoder_hidden, encoder_outputs)
    topv, topi = decoder_output.topk(1)#Take the best one
    #teacher forcing or not
    #decoder_input = topi.squeeze().detach()
    decoder_input = output_tensor[di]#For next round
    decoder_attentions[di] = decoder_attention[0,0]
    decoded_words.append(output_lang.index2word[topi.item()])#append the best token right now
    
    if decoder_input.item() == EOS_token:

        break;

/home/yz4499/miniconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/yz4499/miniconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [73]:
decoder_attentions

tensor([[0.5106, 0.1436, 0.0858, 0.0185, 0.0340, 0.0415, 0.0415, 0.0415, 0.0415,
         0.0415],
        [0.5122, 0.1391, 0.0874, 0.0178, 0.0318, 0.0423, 0.0423, 0.0423, 0.0423,
         0.0423],
        [0.4883, 0.1467, 0.0946, 0.0178, 0.0328, 0.0440, 0.0440, 0.0440, 0.0440,
         0.0440],
        [0.4913, 0.1440, 0.0894, 0.0180, 0.0327, 0.0449, 0.0449, 0.0449, 0.0449,
         0.0449],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000]], grad_fn=<

# Second Beam Search: 

In [74]:
import operator

In [75]:
class SearchNode(object):
    def __init__(self, word_idx, hidden, curr_score, prev, curr_length):
        self.word_idx = word_idx        
        self.hidden = hidden
        self.prev = prev
        if self.prev == None:
            self.score = curr_score
        else:
            self.score = self.prev.score + curr_score#The score summed up so far
        
        self.curr_length = curr_length #the nth node in the sentence
        self.next = None
        



In [299]:
def init_beam(decoder_hidden, encoder_outputs, decoder, k):
    decoder_input = torch.tensor([[SOS_token]], device = device)
    start_node = SearchNode(SOS_token,decoder_hidden, 0,None,1 )
    
    decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
    curr_nodes = []
    scores, indexes = torch.topk(decoder_output, k)##Watch out for dimension, topk on the vocab direction
    for i in range(k):
        curr_idx = indexes[0][i].view(1,-1)
            
        curr_score = scores[0][i].item()
        curr_node = SearchNode(curr_idx, decoder_hidden, curr_score, start_node, 2)

        curr_nodes.append((curr_node, curr_score))
        
    return curr_nodes, decoder_hidden

def beam_search(decoder_hidden,encoder_outputs,decoder,k):
    curr_nodes, decoder_hidden = init_beam(decoder_hidden, encoder_outputs, decoder, k)

    end_nodes = []#List including the ending nodes to trace back
    while len(curr_nodes) != 0 and len(end_nodes) < k:
        next_nodes = []   
        for curr_node, curr_score in curr_nodes:
            #First we decide whether to terminate the search or not
            #print(curr_node.word_idx.item())
            
            if (curr_node.word_idx == EOS_token):#If already at the end of the sentence, then skip the calculation
                end_nodes.append((curr_node, curr_node.score))
                if len(end_nodes) >= k:
                    break;
                else:
                    continue;
                    
            if curr_node.curr_length == max_length:
                end_nodes.append((curr_node, curr_node.score))
                if len(end_nodes) >= k:
                    break;
                else:
                    continue;
            
            decoder_input = curr_node.word_idx
            decoder_hidden = curr_node.hidden
            decoder_output, decoder_hidden,decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            scores, indexes = torch.topk(decoder_output, k)##Watch out for dimension, topk on the vocab direction
            for i in range(k):
                next_idx = indexes[0][i].view(1,-1)
                next_score = scores[0][i].item()
                next_node = SearchNode(next_idx, decoder_hidden, next_score, curr_node, curr_node.curr_length+1)

                next_nodes.append((next_node, next_node.score))
        
        curr_nodes = sorted(next_nodes, key = operator.itemgetter(1))
        curr_nodes = curr_nodes[:k]
        #j+=1

    return end_nodes        

# Disintegration of beam search

In [300]:
#initialize the decoder_hidden here
decoder_input = torch.tensor([[SOS_token]], device = device)
decoder_hidden = encoder_hidden#1x1x256

In [301]:
end_nodes = beam_search(decoder_hidden, encoder_outputs, decoder = attn_decoder, k = 2)

/anaconda/envs/py36/lib/python3.6/site-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [302]:
end_nodes

[(<__main__.SearchNode at 0x122549a90>, -66.93753623962402),
 (<__main__.SearchNode at 0x122549588>, -66.88386154174805)]

In [307]:
#Genereating the sentences.
sentence = []
end_nodes = sorted(end_nodes, key = operator.itemgetter(1),reverse = True)
(end_node, end_score) = end_nodes[0]
while end_node.word_idx != SOS_token:
    sentence.append(output_lang.index2word[end_node.word_idx.item()])
    end_node = end_node.prev
sentence = sentence[::-1]#Reverse the sentence to get the sentence

In [308]:
sentence

['cleared',
 'snakes',
 'singapore',
 'graceful',
 'working',
 'deluding',
 'wild',
 'ashore',
 'dragging']

# End of disintegration

In [ ]:
#Genereating the sentences.
sentence = []
end_nodes = sorted(end_nodes, key = operator.itemgetter(1),reverse = True)
end_node = end_nodes[0]
while end_node.word_idx != EOS_token:
    sentence.append(end_node.word_idx)
    end_node = end_node.prev
sentence = sentence[::-1]#Reverse the sentence to get the sentence

In [224]:
#initialize the decoder_hidden here
decoder_input = torch.tensor([[SOS_token]], device = device)
decoder_hidden = encoder_hidden#1x1x256

In [ ]:
else:
    # Without teacher forcing: use its own predictions as the next input
    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        #topv is the score, topi is the index of the highest score
        topv, topi = decoder_output.topk(1)

        decoder_input = topi.squeeze().detach()  # detach from history as input

        loss += criterion(decoder_output, target_tensor[di])
        if decoder_input.item() == EOS_token:
            break

In [228]:
#Evaluations: beam_width = 2
end_nodes = beam_search(decoder_hidden, encoder_outputs,decoder = attn_decoder, k=2)






/anaconda/envs/py36/lib/python3.6/site-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


AttributeError: 'tuple' object has no attribute 'word_idx'

In [ ]:
#training:
for di in range(max_length):
    decoder_output,decoder-hid